In [63]:
! pwd

/home/changhong/prj/finn/notebooks


In [64]:
import torch.nn.functional as F
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU, QuantIdentity
from brevitas.quant import Int8WeightPerTensorFixedPoint, Int8ActPerTensorFixedPoint
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os


In [65]:
! nvidia-smi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Wed Jun 18 16:55:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 4000 Ada Gene...    Off |   00000000:02:00.0  On |                  Off |
| 30%   32C    P8             11W /  130W |    5453MiB /  20475MiB |     43%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [66]:
# print current working directory
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/changhong/prj/finn/notebooks


In [67]:
notebook_name = "/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST"
finn_root = os.getcwd()
build_dir = finn_root+ notebook_name +"/build"
model_dir = finn_root+ notebook_name +"/model"
data_dir = finn_root+ notebook_name +"/data"

# Create directories if they do not exist
os.makedirs(build_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
print(f"Data directory: {data_dir}")
print(f"Finn root directory: {finn_root}")
print(f"Build directory: {build_dir}")
print(f"Model directory: {model_dir}")

Data directory: /home/changhong/prj/finn/notebooks/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST/data
Finn root directory: /home/changhong/prj/finn/notebooks
Build directory: /home/changhong/prj/finn/notebooks/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST/build
Model directory: /home/changhong/prj/finn/notebooks/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST/model


In [68]:
# Hyperparameters
EPOCHS = 50
LR = 0.001
BATACH_SIZE = 128
RANDOM_SEED = 1998

In [69]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split



# Define data transformations
# Normalize using MNIST mean (0.1307) and std deviation (0.3081)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load full MNIST training set (60,000 samples)
full_train_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=True,       # Load training set (not test set)
    download=True,    # Download if not exists
    transform=transform
)

# Standard academic split (5:1:1 ratio)
# Train: 50,000 | Val: 10,000 | Test: 10,000 (official test set)
train_size = 50000    # 5/6 of training data
val_size = 10000      # 1/6 of training data

# Split the full training set into train/val subsets
train_dataset, val_dataset = random_split(
    full_train_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)  # Ensure reproducibility
)

# Load official test set (10,000 samples)
test_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=False,      # Load test set
    download=True,
    transform=transform
)

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATACH_SIZE,
    shuffle=True      # Shuffle training data
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATACH_SIZE,
    shuffle=False     # No need to shuffle validation data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATACH_SIZE,  # Larger batch for evaluation
    shuffle=False
)

# Print dataset statistics
print("Dataset split complete:")
print(f"Training set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")
print(f"Test set: {len(test_dataset)} samples")


Dataset split complete:
Training set: 50000 samples
Validation set: 10000 samples
Test set: 10000 samples


In [70]:
from torch.nn import Module
import torch.nn as nn
import torch.nn.functional as F
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from brevitas.core.restrict_val import RestrictValueType
from brevitas.core.scaling import ScalingImplType
import torch


total_bits = 8   #width for weights and activations
n = 7            #fractional part
class LeNet5(Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Changhong, add input quant layer
        # self.quant_input = qnn.QuantIdentity(
        #     quant_type=QuantType.INT,
        #     bit_width=total_bits,
        #     max_val=1.0,  # 假设输入数据已归一化到 [0,1]
        #     restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
        #     scaling_impl_type=ScalingImplType.CONST
        # )
        self.quant_input = qnn.QuantIdentity(
            quant_type=QuantType.INT,
            bit_width=8,
            scaling_init=1.0,               # 初始缩放因子
            scaling_impl_type=ScalingImplType.PARAMETER,  # 允许训练中调整
            scaling_per_output_channel=False,  # 对于单通道输入，通常不需要每个输出通道的缩放
            )


        self.conv1 = qnn.QuantConv2d(in_channels= 1,
                                     out_channels= 20,
                                     kernel_size= 3,
                                     padding= 1,
                                     bias= False,
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width= total_bits,
                                     weight_restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                     weight_scaling_impl_type=ScalingImplType.CONST,
                                     weight_scaling_const=1.0)
        self.relu1 = qnn.QuantReLU(quant_type=QuantType.INT, 
                                   bit_width=8, 
                                   max_val= 1- 1/128.0,
                                   restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                   scaling_impl_type=ScalingImplType.CONST )

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        

        self.conv2 = qnn.QuantConv2d(in_channels= 20,
                                     out_channels= 50,
                                     kernel_size= 3,
                                     padding= 1,
                                     bias= False,
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8,
                                     weight_restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                     weight_scaling_impl_type=ScalingImplType.CONST,
                                     weight_scaling_const=1.0 )

        self.relu2 = qnn.QuantReLU(quant_type=QuantType.INT, 
                                   bit_width=8, 
                                   max_val= 1- 1/128.0,
                                   restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                   scaling_impl_type=ScalingImplType.CONST )

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        """
        # for 32-bit precision FC layers
        self.fc1   = nn.Linear(7*7*50, 500)

        self.relu3 = nn.ReLU()

        self.fc2   = nn.Linear(500,10)

        """
        

        # for fixed-point precision FC layers
        self.fc1   = qnn.QuantLinear(7*7*50, 500,
                                     bias= True,
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=32,
                                     weight_restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                     weight_scaling_impl_type=ScalingImplType.CONST,
                                     weight_scaling_const=1.0)
        

        self.relu3 = qnn.QuantReLU(quant_type=QuantType.INT, 
                                   bit_width=8, 
                                   max_val= 1- 1/128.0,
                                   restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                   scaling_impl_type=ScalingImplType.CONST )

        self.fc2   = qnn.QuantLinear(500, 10,
                                     bias= True,
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8,
                                     weight_restrict_scaling_type=RestrictValueType.POWER_OF_TWO,
                                     weight_scaling_impl_type=ScalingImplType.CONST,
                                     weight_scaling_const=1.0)
    def forward(self, x):
        out = self.quant_input(x)  # Apply input quantization
        out = self.relu1(self.conv1(out))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.relu3(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out, dim=1)
        return out

In [71]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

def train_model(model, train_loader, val_loader, test_loader, num_epochs=10, lr=0.001):
    # Initialize device (use GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training phase with progress bar
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for inputs, labels in train_progress:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_progress.set_postfix({
                'loss': running_loss/(train_progress.n+1),
                'acc': 100.*correct/total
            })
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        print(f'Validation - Loss: {val_loss/len(val_loader):.4f}, Acc: {100.*val_correct/val_total:.4f}%')
    
    # Final test evaluation
    model.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f'Test Accuracy: {100.*test_correct/test_total:.4f}%')
    
    return model




In [72]:
model = LeNet5()
trained_model = train_model(model, train_loader, val_loader, test_loader, num_epochs=EPOCHS, lr=LR)

Epoch 1/50 [Train]:   0%|                                                                                                             | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  return F.conv2d(input, weight, bias, self.stride,
/home/changhong/prj/finn/deps/brevitas/src/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  output_tensor = linear(x, quant_weight, quant_bias)
Epoch 1/50 [Train]: 100%|█████████████████████████████████████████████████████████████████████████████| 391/391 [00:04<00:00, 90.52it/s, loss=0.218, ac

Validation - Loss: 0.0708, Acc: 97.7600%


Epoch 2/50 [Train]: 100%|█████████████████████████████████████████████████████████████████████████████| 391/391 [00:04<00:00, 93.32it/s, loss=0.052, acc=98.6]


Validation - Loss: 0.0460, Acc: 98.5600%


Epoch 3/50 [Train]: 100%|████████████████████████████████████████████████████████████████████████████| 391/391 [00:04<00:00, 90.73it/s, loss=0.0334, acc=99.1]


...

Validation - Loss: 0.0392, Acc: 99.3200%


Epoch 49/50 [Train]: 100%|███████████████████████████████████████████████████████████████████████████| 391/391 [00:04<00:00, 92.66it/s, loss=1.78e-7, acc=100]


Validation - Loss: 0.0396, Acc: 99.3100%


Epoch 50/50 [Train]: 100%|███████████████████████████████████████████████████████████████████████████| 391/391 [00:04<00:00, 93.04it/s, loss=1.49e-7, acc=100]


Validation - Loss: 0.0402, Acc: 99.3200%
Test Accuracy: 99.1300%


In [73]:
# save the trained model weights
model_path = os.path.join(model_dir, "lenet_mnist_int8.pth")
torch.save(trained_model.state_dict(), model_path)
print(f"Trained model saved to {model_path}")

# save the model to ONNX format
onnx_model_path = os.path.join(model_dir, "lenet_mnist_int8.onnx")

# put model back to CPU for ONNX export
trained_model.to('cpu')
torch.onnx.export(
    trained_model,
    torch.randn(1, 1, 28, 28),  # Dummy input for ONNX export
    onnx_model_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output']
)
print(f"ONNX model saved to {onnx_model_path}")

Trained model saved to /home/changhong/prj/finn/notebooks/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST/model/lenet_mnist_int8.pth


/home/changhong/prj/finn/deps/brevitas/src/brevitas/quant_tensor/int_quant_tensor.py:32: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  signed = torch.tensor(signed, dtype=torch.bool)
/home/changhong/prj/finn/deps/brevitas/src/brevitas/quant_tensor/int_quant_tensor.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  training = torch.tensor(training, dtype=torch.bool)
/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference 

ONNX model saved to /home/changhong/prj/finn/notebooks/EF-US-Engine-Free-Unstructured-Sparsity-Design-Alleviates-Accelerator-Bottlenecks/casestudy1/LeNet_MNIST/model/lenet_mnist_int8.onnx


/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:687: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:1178: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_graph_shape_type_inference(
